In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os

# set the right device
#os.environ['CUDA_VISIBLE_DEVICES'] = '0'
# NOTE: assuming we are in `ca_body/notebooks`
sys.path.insert(0, '../')
from attrdict import AttrDict

import torch as th
import numpy as np

from omegaconf import OmegaConf
from torchvision.utils import make_grid

from ca_body.utils.module_loader import load_from_config
from ca_body.utils.lbs import LBSModule
from ca_body.utils.image import linear2displayBatch
from ca_body.utils.geom_body import EulerXYZ_to_matrix

from ca_body.utils.train import (
    load_from_config,
    load_checkpoint,
    build_optimizer,
    train,
)
from ca_body.utils.dataloader import BodyDataset, worker_init_fn
from ca_body.utils.torchutils import to_device
from torch.utils.data import DataLoader

from ipywidgets import interact


device = th.device('cuda:0')

In [13]:
config_path = '../config/mesh_vae_example.yml'
config = OmegaConf.load(config_path)
config.root_dir = '../'

In [14]:
device = th.device(f"cuda:0")

train_dataset = BodyDataset(**config.data)
static_assets = AttrDict(train_dataset.static_assets)

model = load_from_config(config.model, assets=static_assets).to(device)
optimizer = build_optimizer(config.optimizer, model)

[2024-04-28 13:56:11][INFO][ca_body.utils.geom]:impainting index image might take a while for sizes >= 1024
[2024-04-28 13:56:15][INFO][ca_body.models.mesh_vae]:ConvDecoder: n_channels = [64, 32, 16, 8, 4]
[2024-04-28 13:56:16][WARNING][ca_body.nn.color_cal]:Requested color-calibration identity camera not present, defaulting to 400870.


In [15]:
floor_Rt = th.as_tensor(static_assets.floor_Rt, device=device)
floor_Rt_inv = th.as_tensor(static_assets.floor_Rt_inv, device=device)

In [16]:
train_loader = DataLoader(
    train_dataset, 
    **config.dataloader,
    worker_init_fn=worker_init_fn,
)

batch = to_device(next(iter(train_loader)), device)

In [17]:
model.cal_enabled = False
model.learn_blur_enabled = False
model.pixel_cal_enabled = False
model.rendering_enabled = True
model.eval();

In [18]:
def update_rendering(cam_x=0, cam_y=1000, cam_z=3000, angle_X=0, angle_Y=-0.91, fidx=0):
    
    angle_Z = -3.14

    B = batch['pose'].shape[0]
    # TODO: should we be able to switch this?
    K = to_device(batch['K'][fidx:fidx+1], device)
    pose = to_device(batch['pose'][fidx:fidx+1], device)
    
    floor_R, floor_t = floor_Rt[:3, :3], floor_Rt[:3, 3]

    ambient_occlusion = to_device(batch['ambient_occlusion'][fidx:fidx+1], device)
    registration_vertices = to_device(batch['registration_vertices'][fidx:fidx+1], device)

    XYZ = th.as_tensor([angle_X, angle_Y, angle_Z], dtype=th.float32, device=device)[np.newaxis].expand(1, -1)
    R = EulerXYZ_to_matrix(XYZ)
    Rt = th.eye(3, 4, device=device)[np.newaxis].expand(1, -1, -1).clone()
    # NOTE: we are rotating around the model space!
    Rt[:,:3,:3] = R
    Rt[:,0,3] = cam_x
    Rt[:,1,3] = cam_y
    Rt[:,2,3] = cam_z

    np_Rt = Rt[0].cpu().numpy()
    XYZ_str = np.array2string(XYZ[0].cpu().numpy(), formatter={'float_kind':lambda x: "%.3f" % x}, separator=',')
    cam_xyz_str = np.array2string(Rt[0,:3,3].cpu().numpy(), formatter={'float_kind':lambda x: "%.0f" % x}, separator=',')

    frame_id_str = str(int(batch['frame_id'][fidx]))
    params_text.value = f'XYZ = {XYZ_str} \ncam_xyz = {cam_xyz_str}\nframe = {frame_id_str}'

    with th.no_grad():
        floor_R_inv, floor_t_inv = floor_Rt_inv[:3, :3], floor_Rt_inv[:3, 3]
        
        campos = (th.bmm(Rt[:, :3,:3].permute(0, 2, 1), -Rt[:, :3, 3:])[...,0]).to(th.float32)
        campos = ((floor_R_inv @ campos.to(th.float64).T).T + floor_t_inv).to(th.float32)        
        
        preds = model(
            pose=pose,
            registration_vertices=registration_vertices,
            ambient_occlusion=ambient_occlusion,
            campos=campos,
            K=K,
            Rt=Rt,
        )    

        geom = preds['geom']
        tex_rec = preds["tex_rec"]

        # transform to socio space
        geom = (th.matmul(geom.to(th.float64), floor_R.T[np.newaxis]) + floor_t).to(th.float32)

        # rendering avatar
        renders = model.renderer(
            geom,
            tex_rec,
            K=K,
            Rt=Rt,
        )    

        np_image = linear2displayBatch(renders['render']).permute(0, 2, 3, 1)[0].cpu().numpy()
        canvas.put_image_data(np_image)        

In [19]:
from ipycanvas import Canvas
from ipywidgets import VBox, HBox, interactive, Output, Textarea

params_text = Textarea(width=1000)

canvas = Canvas(width=model.renderer.w, height=model.renderer.h)

B = batch["pose"].shape[0]

controls = interactive(
    update_rendering, 
    cam_z=(-5000, 3000, 5), 
    cam_y=(-2000, 3000, 5),
    cam_x=(-3000, 3000, 5),     
    angle_X=(-np.pi, np.pi, (np.pi / 180.0) * 1.0),
    angle_Y=(-2 * np.pi, 2 * np.pi, (np.pi / 180.0) * 1.0),    
    fidx=(0, B-1),
)
# TODO: add rendering?
from ipywidgets import HBox, Label
style = {"description_width": "initial"}
from ipywidgets import IntSlider

HBox([canvas, VBox([controls, params_text])])